In [20]:
import glob
s
import numpy as np
import cv2 as cv
from skimage.morphology import skeletonize


from plantcv.plantcv.morphology import prune

from plantcv.plantcv import fill, fill_holes
import networkx as nx

from sknw import build_sknw
from sklearn.metrics.pairwise import euclidean_distances

In [22]:
def get_skeleton(src):
    gray = cv.cvtColor(src, cv.COLOR_BGR2GRAY) # переход к полутоновому изображению
    hsv = cv.cvtColor(src, cv.COLOR_BGR2HSV)

    lower_blue = np.array([ 70, 45, 94])
    upper_blue = np.array([120, 200, 255])
    mask = cv.inRange(hsv, lower_blue, upper_blue)
    #plt.imshow(mask)
    #plt.show()

    mask = np.clip(mask, 0, 1).astype(np.uint8)
    kernel = cv.getStructuringElement(cv.MORPH_ELLIPSE, (3,3))
    tmp = cv.erode(mask,  kernel, iterations=1)
    kernel = cv.getStructuringElement(cv.MORPH_ELLIPSE, (10,10))
    tmp = cv.dilate(tmp,  kernel, iterations=5)

    tmp = skeletonize(np.clip(tmp, 0, 1)).astype(np.uint8)
    tmp, _, _ = prune(skel_img=tmp, size=int(110))
    return tmp

In [23]:
from sklearn.metrics.pairwise import euclidean_distances
def merge_equal_points(points, min_dist=100):
    pdist = euclidean_distances(points)
    merged_points_mask = np.zeros(len(pdist), dtype=bool)
    for i in range(len(pdist)):
        if merged_points_mask[i]: continue
        merged_points_mask |= ((pdist[i] <= min_dist) & (pdist[i] != 0.0))
    return points[~merged_points_mask]

In [27]:
def dfs(time, circle, src, visited,height,width):
    cx = int(circle[0]) # координата x центра вершины
    cy = int(circle[1]) # координата y центра вершины
    todo = [(cy,cx)]
    degree = 0 # искомая степень вершины
    while todo:
        y,x = todo.pop() # берем координаты пикселя с вершины стека

        # ходим только по белым пикселям, которые лежат в пределах картинки и не были посещены ранее в этом же dfs
        if not (0 <= y < height) or not (0 <= x < width) or (src[y,x] == 0)  or (visited[y,x] == time):
            continue

        visited[y,x] = time # помечаем пиксель как посещенный 
        dist = ((cx - x)**2 + (cy - y)**2) ** 0.5
        if dist > 55:
            R = 5

            # помечаем все пиксели в радиусе R как посещенные
            cv.circle(visited, (x,y), R, time, -1) 
            degree+=1 
            continue
        
        for dy in range(-1, 2):
            for dx in range(-1, 2):
                ny = y + dy
                nx = x + dx
                todo += [(ny, nx)]
    return degree

In [28]:
def get_description_graph(src):
    skeleton = get_skeleton(src)
    #plt.imshow(skeleton, cmap='gray')
    #plt.show()
    # строит из скелета граф из networkx
    g = build_sknw(skeleton, ring=False, iso=False)
    nodes = g.nodes()
    # y, x
    circles = np.array([nodes[i]['o'] for i in nodes])
    # x, y
    circles[:, [0, 1]] = circles[:, [1, 0]]
    # убираем побочные точки
    circles = merge_equal_points(circles, min_dist=45)
    for i in circles:
           cv.circle(skeleton, (i[0],i[1]), 3, 1, -1)

    # массив, где будем хранить посещенные вершины
    visited = np.zeros(skeleton.shape, np.uint8) 

    # ширина и высота изображения
    width, height = skeleton.shape[1], skeleton.shape[0] 

    # массив, в котором k-я компонента есть число вершин степени k в представленном графе
    ans = np.zeros(10, np.uint8) 

    # текущее "время". Нужно, чтобы мы не могли ходить по одним и тем же пикселям в вызове одного dfs'а, но могли ходить по одним и тем же пикселям в разных dfs'ах
    time = 1

    #идем по всем врешинам, считаем их степени в dfs
    for i in circles:
        
        # начинаем поиск в глубину от центра вершины. Функция возвращает степень текущей вершины
        cnt = dfs(time, i, skeleton, visited, height, width)
        ans[cnt]+=1 # обновляем ответ
        time += 1 # обновляем "время
    return ans[3:5]

In [29]:
arr = glob.glob('images/Expert/*.jpg')

# словарь для классов графа
graph_class = {'[3 3]': [], 
                '[4 1]': [], 
                '[5 2]': [], 
                '[4 2]': []}
for el in arr:
    print('processing of image ' + el)

    # считываем изображение
    src = cv.imread(el) 

    # находим массив со всеми степенями вершин
    ans = get_description_graph(src) 

    # для использования массива как ключа в словаре, переводим его в сторку
    ans_str = str(ans) 
    if ans_str not in graph_class:

        # создаем новый список под новый класс
        graph_class[ans_str] = list() 
        
    # добавляем граф в нужный класс
    graph_class[ans_str].append(el) 

# вывод ответа
print('\nANSWER:')
num_class = 0
for el in graph_class.values():
    num_class += 1
    print("CLASS", str(num_class) + ":", el)

processing of image images/Expert/9.jpg
processing of image images/Expert/28.jpg
processing of image images/Expert/29.jpg
processing of image images/Expert/16.jpg
processing of image images/Expert/10.jpg
processing of image images/Expert/22.jpg
processing of image images/Expert/23.jpg
processing of image images/Expert/18.jpg

ANSWER:
CLASS 1: ['images/Expert/16.jpg', 'images/Expert/18.jpg']
CLASS 2: ['images/Expert/28.jpg', 'images/Expert/29.jpg']
CLASS 3: ['images/Expert/22.jpg', 'images/Expert/23.jpg']
CLASS 4: ['images/Expert/9.jpg', 'images/Expert/10.jpg']
